In [58]:
#Import packages
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import datetime as dt
import numpy as np
from scipy import stats
import math as m
from matplotlib import pyplot as plt
from scipy.stats import norm
#from arch import arch_model 

In [59]:
#Loading data

filename='DataLab2.csv'
table = pd.read_csv(filename)
table['Date'] = pd.to_datetime(table['Date'])
table = table.set_index('Date')
start = dt.datetime(2005, 1, 1)
end = dt.datetime(2008,12,31)
backtesting_sample = table[start:end]
backtesting_sample['Losses'] = backtesting_sample['PL']*-1

print(backtesting_sample)

              PL  VaRBHS      VaREWMA         VaRn         VaRt       VaRPot  \
Date                                                                           
2005-01-01   190     NaN          NaN          NaN          NaN          NaN   
2005-02-01   190     NaN          NaN          NaN          NaN          NaN   
2005-03-01  -220     NaN          NaN          NaN          NaN          NaN   
2005-05-01   120     NaN          NaN          NaN          NaN          NaN   
2005-06-01   450     NaN          NaN          NaN          NaN          NaN   
...          ...     ...          ...          ...          ...          ...   
2008-12-25   500  3835.0  5685.328442  3099.600997  3894.837749  5533.676874   
2008-12-26   300  3835.0  5651.307303  3099.176876  3885.509964  5533.676874   
2008-12-28  -680  3835.0  5493.420633  3098.836653  3882.021708  5533.676874   
2008-12-29 -1690  3835.0  5330.700046  3101.208561  3869.046817  5533.676874   
2008-12-31   670  3835.0  5206.385930  3

In [60]:
#I choose start and end dates so that we have exactly M = 250 of the latest trading days in accordance with Basel
start2007 = dt.datetime(2007, 4, 1) 
end2007 = dt.datetime(2007,12,31)

start2008 = dt.datetime(2008, 4, 1)
end2008 = dt.datetime(2008,12,31)

sample_2007 = backtesting_sample[start2007:end2007]
sample_2008 = backtesting_sample[start2008:end2008]

print(len(sample_2007))
print(len(sample_2008))


250
250


In [61]:
#creating dataframe storing pvalues Kupiec

columns = ['VaRBHS', 'VaREWMA', 'VaRn', 'VaRt', 'VaRPot'] 

#Kupiec
Kupiec_pval = pd.DataFrame()
Kupiec = pd.DataFrame()

#Traffic light
Violations = pd.DataFrame()
Traffic_light = pd.DataFrame()

#Christoffersen
Christoffersen_pval = pd.DataFrame()
Christoffersen_frame = pd.DataFrame()


In [62]:
def Christoffersen(violations): #Used for christoffersen test
    n1 = sum(violations)
    n0 = len(violations) - n1

    n11 = 0
    n00 = 0
    n01 = 0
    n10 = 0

    for r in range(0,len(violations)-1):
        if violations[r]==1 and violations[r+1]==1:
            n11 += 1
        if violations[r]==0 and violations[r+1]==0:
            n00 += 1
        if violations[r]==0 and violations[r+1]==1:
            n01 += 1
        if violations[r]==1 and violations[r+1]==0:    
            n10 += 1
    # Empirical estimates of all needed probabilities
    pi00 = n00 / (n00+n01) # Slide 8 VL 9
    pi01 = n01 / (n00+n01)
    pi10 = n10 / (n10+n11)
    pi11 = n11 / (n10+n11)
    pi0 = n0 / (n1+n0)
    pi1 = n1 / (n1+n0)     


    lnNull = np.log(pi0**n0*pi1**n1) # Slide 9 VL 9, second equation 
    lnAlt = np.log(pi00**n00*pi01**n01*pi10**n10*pi11**n11) # Slide 9 VL 9, first equation
    LRind = -2*(lnNull-lnAlt)

    # Pvalue for test stat that is chisquare(1) distributed (upper since right tail for one-sided test)
    pVal = 1-stats.chi2.cdf(LRind,1) # 1 minus since right tail for one-sided test

    return(pVal) #pVal for test


In [63]:
#Variables
alpha = 0.99 #99% confidence level as with Basel
M = 250 #according to Basel, we could of course set M = len(sample) in for loop below as well

In [64]:
#Backtesting VaR


for column in columns: #for each VaR measurement
    #Temporary vectors for storing values in dataframes
    Kup_pval = [] #Kupiec pvals
    Kup_bool = [] #Kupiec reject or keep null hypothesis

    Chris_pval = [] #Christoffersen pvals
    Chris_bool = [] #Christoffersen reject or keep null hypothesis

    zone = []  #traffic lights: green, amber or red
    nrval_vec = [] #nr of violations vector

    for sample in [sample_2007, sample_2008]:
        violations = sample['Losses']>sample[column] # violations is when loss > VaR
        violations = violations.astype(int) #convert to vector of 0s and 1s
    

        n_violations = sum(violations) #number of violations
        nrval_vec.append(n_violations) #save in vec

        #Kupiec test
        pval_Kupiec = 1-stats.binom.cdf(n_violations-1,M,1-alpha) # Following slide 13 Video Lecture 8, one sided test
        Kup_pval.append(pval_Kupiec)
        Kup_bool.append(pval_Kupiec > 1-alpha)

        #Christoffersen test
        pval_Chris = Christoffersen(violations) #Function defined in cell above
        Chris_pval.append(pval_Chris)
        Chris_bool.append(pval_Chris > 1-alpha)

        #Traffic light test
        if n_violations >= 10:
            zone.append('Red')
        elif n_violations >= 5:
            zone.append('Amber')
        else:
            zone.append('Green')
            
    #adding values to dataframes
    Kupiec_pval[column] = Kup_pval
    Kupiec[column] = Kup_bool

    Christoffersen_pval[column] = Chris_pval
    Christoffersen_frame[column] = Chris_bool

    Violations[column] = nrval_vec
    Traffic_light[column] = zone

    #setting index to all dataframes
    Kupiec_pval.index = ['2007', '2008']
    Kupiec.index = ['2007', '2008']
    Christoffersen_pval.index = ['2007', '2008']
    Christoffersen_frame.index = ['2007', '2008']
    Violations.index = ['2007', '2008']
    Traffic_light.index = ['2007', '2008']

print('Expected nr. of violations = 0.01*250 = 2.5 \n')
print('number of violations \n')
print(Violations)
print('\n \n' +'P-values using Kupiec for each year and VaR \n')
print(Kupiec_pval)
print('\n' + 'Kupiec Test: If false then we have a bad VaR estimate (underestimating) and we reject H_0 \n')
print(Kupiec)
print('\n \n' + 'P-values using Christoffersen for each year and VaR \n')
print(Christoffersen_pval)
print('\n' + 'Christoffersen Test: If false then we have a bad VaR estimate  (underestimating) and we reject H_0 \n')
print(Christoffersen_frame)
print('\n ' + 'Traffic Light test \n')
print(Traffic_light)



Expected nr. of violations = 0.01*250 = 2.5 

number of violations 

      VaRBHS  VaREWMA  VaRn  VaRt  VaRPot
2007      15        5    36    27       5
2008       7        6    12     9       3

 
P-values using Kupiec for each year and VaR 

            VaRBHS   VaREWMA      VaRn      VaRt    VaRPot
2007  5.126074e-08  0.107812  0.000000  0.000000  0.107812
2008  1.370145e-02  0.041183  0.000011  0.001057  0.456831

Kupiec Test: If false then we have a bad VaR estimate (underestimating) and we reject H_0 

      VaRBHS  VaREWMA   VaRn   VaRt  VaRPot
2007   False     True  False  False    True
2008    True     True  False  False    True

 
P-values using Christoffersen for each year and VaR 

        VaRBHS   VaREWMA      VaRn      VaRt    VaRPot
2007  0.256077  0.001622  0.019192  0.015718  0.620319
2008  0.009151  0.004223  0.012301  0.030276  0.019575

Christoffersen Test: If false then we have a bad VaR estimate  (underestimating) and we reject H_0 

      VaRBHS  VaREWMA  VaRn  V

In [65]:
#Now create dataframes for backtesting ES
ES_Z = pd.DataFrame()
Backtest_ES = pd.DataFrame()

In [66]:
VaR_list = ['VaREWMA', 'VaRn', 'VaRt', 'VaRPot']
ES_list = ['ESEWMA', 'ESn', 'ESt', 'ESPot']

for i in range(len(VaR_list)): #Same as for VaR but now for backtesting ES
    #Temporary vectors
    Z_vec = [] #z-values 
    bool_vec = [] #reject or keep H_0

    for sample in [sample_2007, sample_2008]:
        violations = sample['Losses']>sample[VaR_list[i]] 
        violations = violations.astype(int)

        #Z-score according to Acerbi-Szekely (2015)
        Z = -M**-1*sum((violations*sample['Losses']/(1-alpha))/sample[ES_list[i]]) + 1

        Z_vec.append(Z) #store z-val
        bool_vec.append(Z > -0.82) #assuming t-dist with dgf = 3, mean = 0 and sigma = 1, see L10 slide 11

       
    ES_Z[ES_list[i]] = Z_vec
    Backtest_ES[ES_list[i]] = bool_vec

    ES_Z.index = ['2007', '2008']
    Backtest_ES.index = ['2007', '2008']

        
print('Z-values backtesting ES for each year and VaR \n')
print(ES_Z)
print('\n' + 'If false then our ES estimate is bad and we reject the null hypothesis that it\'s a good estimate \n')
print(Backtest_ES)


      


Z-values backtesting ES for each year and VaR 

        ESEWMA        ESn       ESt     ESPot
2007 -0.477792 -18.617657 -1.324354 -1.184306
2008 -1.065068  -6.638042  0.103399 -0.514151

If false then our ES estimate is bad and we reject the null hypothesis that it's a good estimate 

      ESEWMA    ESn    ESt  ESPot
2007    True  False  False  False
2008   False  False   True   True


a) There is a lot of variation depending on the year for the different estimates. Understandable since 2008 was a hectic year. For VaR estimates they are consistent in each individual test except for VaRBHS but they typically differ between test. The only VaR estimate that is consistent for both years and Christoffersen and Kupiec is VaRPOT.

No ES method is consistent for both 2007 and 2008.

b) For VaR I would pick VaRPOT since it passes both Christoffersen and Kupiec test. In Traffic light test it is Amber / Green.

ES is a bit more tricky, as there doesn't seem to be a perfect method. I would probably pick either ESEWMA or ESPoT since they have the highest Z-values. For consistency sake I would choose ESPoT since it fits nicely with VaRPOT but I don't think there is a perfect answer in this case.